# Large Model 을 accelerate를 이용해서 loading하는 예시. 

offloading by cpu, disk

In [1]:
import os
import torch
print(os.environ['TRANSFORMERS_CACHE'])

/data/huggingface-cache


In [2]:
from huggingface_hub import hf_hub_download

checkpoint = 'facebook/opt-2.7b'
weights_path = hf_hub_download(checkpoint, 'pytorch_model.bin', cache_dir=os.environ['TRANSFORMERS_CACHE'])
print(weights_path)

/data/huggingface-cache/models--facebook--opt-2.7b/snapshots/397f71a473a150c00f0fe3fc4a2f78ff3ccaf82d/pytorch_model.bin


In [8]:
import os
from accelerate import init_empty_weights, dispatch_model, infer_auto_device_map, load_checkpoint_and_dispatch, load_checkpoint_in_model, dispatch_model
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
config = AutoConfig.from_pretrained(checkpoint)

with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)
model.tie_weights()
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2560, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2560)
      (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
          (final_layer_norm): Laye

In [12]:
max_mem = 2686198491 # 2G

device_map = infer_auto_device_map(
    model.model, 
    #max_memory={0: max_mem, 'cpu': max_mem*2},
    max_memory={0: max_mem},
    no_split_module_classes=["OPTDecoderLayer"], 
    dtype='float16'
)

print(device_map)

{'decoder.embed_tokens': 0, 'decoder.embed_positions': 0, 'decoder.final_layer_norm': 0, 'decoder.layers.0': 0, 'decoder.layers.1': 0, 'decoder.layers.2': 0, 'decoder.layers.3': 0, 'decoder.layers.4': 0, 'decoder.layers.5': 0, 'decoder.layers.6': 0, 'decoder.layers.7': 0, 'decoder.layers.8': 0, 'decoder.layers.9': 0, 'decoder.layers.10': 0, 'decoder.layers.11': 0, 'decoder.layers.12': 0, 'decoder.layers.13': 0, 'decoder.layers.14': 'disk', 'decoder.layers.15': 'disk', 'decoder.layers.16': 'disk', 'decoder.layers.17': 'disk', 'decoder.layers.18': 'disk', 'decoder.layers.19': 'disk', 'decoder.layers.20': 'disk', 'decoder.layers.21': 'disk', 'decoder.layers.22': 'disk', 'decoder.layers.23': 'disk', 'decoder.layers.24': 'disk', 'decoder.layers.25': 'disk', 'decoder.layers.26': 'disk', 'decoder.layers.27': 'disk', 'decoder.layers.28': 'disk', 'decoder.layers.29': 'disk', 'decoder.layers.30': 'disk', 'decoder.layers.31': 'disk'}


In [5]:
!mkdir ../offload_folder

offload_folder = os.path.join(os.getcwd(), 'offload_filder')
offload_folder

mkdir: cannot create directory ‘../offload_folder’: File exists


'/data/workspace/offload_filder'

In [6]:
load_checkpoint_and_dispatch(
    model.model, 
    weights_path, 
    #device_map="auto",
    device_map=device_map, 
    offload_folder=offload_folder, 
    dtype='float16', 
    offload_state_dict=True
)
model.tie_weights()

inputs = tokenizer("Hugging Face is pushing the convention that a unicorn with two horns becomes a llama.", return_tensors="pt")
output = model.generate(inputs["input_ids"].to(0), max_length=50, do_sample=True)

print(tokenizer.decode(output[0].tolist()))

ValueError: model.decoder.embed_tokens.weight doesn't have any device set.